### Run models
#### List of tasks accomplished in this Jupyter Notebook:
- Run simulation tasks using empirical data from starved animals
- Run simulation tasks using empirical data from fed animals

In [ ]:
import numpy as np
import pandas as pd
import math, random
import time as ti
from copy import deepcopy

t0 = ti.time()
print(ti.strftime("%a, %d %b %Y %H:%M", ti.gmtime()))

In [ ]:
# DECLARE CONSTANT VARIABLES USED IN ALL SIMULATIONS

widths = np.arange(50, 201, 10) # Container widths to be tested, in mm. 
n_trials = 1000 # Number of simulations (each simulation starts in a randomized location)

# DECLARE ADDITIONAL WIDTHS FOR REVIEWER COMMENTS
widths = [500, 1000, 1500]
n_trials = 1000

In [ ]:
# SIMULATED CHEMOKINESIS FUNCTION

def chemokinesis(movedf, width, radius_food, food_x, food_y, origin):
    '''
    Explore the arena until finding the food source. 
    Returns time spent searching (in seconds) and trajectory path (xlist, ylist)
    
    Model changes speed based on absolute concentration. 
    '''
    # Split the given speedlist into half by speed
    movedf_sorted = movedf.sort_values(by=["speed"], axis=0, ascending=True)
    movedf_sorted.reset_index(inplace=True)
    split_index = int(len(movedf_sorted)/2)
    slow_df = movedf_sorted.iloc[0:split_index].copy()
    fast_df = movedf_sorted.iloc[split_index:].copy()

    finish_check = False # Check if the food has been reached
    valid_check = False # Check if current position is within arena bounds
    
    if origin == None:
        while valid_check == False:
            x0, y0 = np.random.uniform(0, width), np.random.uniform(0, width)
            valid_check = check_if_valid(x0, y0, width)
    else:
        x0, y0 = origin[0], origin[1]

    xlist, ylist, prev_angle = [x0], [y0], random.uniform(0, 360)
    finish_check = check_if_done(x0, y0, radius_food, food_x, food_y)
    
    # Continue moving until food has been found
    while finish_check == False:
        valid_check = False
        
        curr_concentration = get_concentration(x0, y0, width/2, radius_food, food_x, food_y)
        p = get_prob(curr_concentration)
        
        while valid_check == False:
            p_rand = np.random.uniform(0, 100)
            if p_rand <= p:
                rand = slow_df.sample(n=1)
                stepsize = rand['speed'].values[0]
                angsize = rand['angle_delta'].values[0]
                angle = get_angle(angsize, prev_angle)
            else:
                rand = fast_df.sample(n=1)
                stepsize = rand['speed'].values[0]
                angsize = rand['angle_delta'].values[0]
                angle = get_angle(angsize, prev_angle)

            x1, y1 = get_next_xy(x0, y0, angle, stepsize)
            valid_check = check_if_valid(x1, y1, width)

        xlist.append(x1)
        ylist.append(y1)

        x0, y0 = deepcopy(x1), deepcopy(y1) # update saved previous point 
        prev_angle = deepcopy(angle)
        curr_concentration = get_concentration(x0, y0, width/2, radius_food, food_x, food_y)
        finish_check = check_if_done(x0, y0, radius_food, food_x, food_y)
        
    return(len(xlist)/2, xlist, ylist)

In [ ]:
# HELPER FUNCTIONS FOR CHEMOKINESIS MODEL

def check_if_valid(x, y, width):
    ''' Returns bool whether or not this point is valid '''
    radius = width/2
    leg1 = np.abs(radius-y)
    leg2 = np.abs(radius-x)
    distance = np.hypot(leg1, leg2)
    if distance > radius:
        return False
    return True

def get_prob(c):
    ''' Returns the probability of choosing the preferred dataset
        as a function of concentration. 
    '''
    c = max(1e-12, c)
    c = min(100-1e-12, c)
    p = 100/(1+np.exp(-0.25*(c-50)))
    return p

def get_angle(angsize, prev_angle):
    ''' Returns random angle between 0 and 360 '''
    return prev_angle + angsize

def get_next_xy(x0, y0, angle, stepsize):
    ''' Return next point as a function of angle and speed '''
    radian = np.deg2rad(angle)
    x1 = x0 + stepsize * math.cos(radian)
    y1 = y0 + stepsize * math.sin(radian)
    return x1, y1

def check_if_done(x0, y0, radius_food, food_x, food_y):
    ''' Returns true if the animal has found the food '''
    leg1 = np.abs(food_x-x0)
    leg2 = np.abs(food_y-y0)
    distance = np.hypot(leg1, leg2)
    if distance <= radius_food:
        return True
    return False

def get_concentration_based_on_distance(distance, a, b):
    ''' Use best fit line to return concentration estimate '''
    if distance <= 0:
        c = 100
    elif distance <= np.hypot(80, 30):
        c = np.exp(b)*np.exp(a*distance)
    else:
        c = 0
    return(c)

def get_concentration(x, y, radius, radius_food, food_x, food_y, 
                      a=-0.08046986117494856, b=4.897119830335053):
    leg1 = np.abs(food_x-x)
    leg2 = np.abs(food_y-y)
    distance = np.hypot(leg1, leg2)
    distance = distance-radius_food 
    c = get_concentration_based_on_distance(distance, a, b)
    return(c)

def get_radius_food(width, perc_area=0.03):
    '''
    perc_area = proportion of arena that should be taken up by food. width = width of arena. 
    '''
    r = width/2
    arena_a = math.pi * r * r
    food_a = arena_a * perc_area
    food_r = np.sqrt(food_a / math.pi)
    return food_r

def run_simulations(n_trials, widths, savename, movedf, function, origin=None):
    # Initialize DF to save values into
    colnames = [str(int(x)) for x in widths]
    rownames = np.arange(0, n_trials)
    rownames = [str(int(x))+'_trial' for x in rownames]
    df_out = pd.DataFrame(index=rownames, columns=colnames)
    
    for width in widths: 
        tlist = []
        radius_food = get_radius_food(width)
        food_x = width/2 # x position of food center
        food_y = width/2 # y position of food center

        for n in range(n_trials):
            timecount, xlist, ylist = function(movedf, width, radius_food, food_x, food_y, origin)
            tlist.append(timecount)

        df_out[str(width)] = tlist

    df_out.to_csv(savename, index=False)
    df_out.head()

- Run simulation tasks using empirical data from starved animals

In [ ]:
df = pd.read_csv("./data/trajectories/summary/modeling_Starved_acclimate_all_animals.csv")
df.dropna(subset=['angle_delta', 'speed_mm_s'], inplace=True, axis=0)
df['speed'] = df['speed_mm_s']/2
print("Mean angle delta:", round(np.mean(df['angle_delta'].tolist()), 2))
print("Mean speed (mm/f):", round(np.mean(df['speed'].tolist()), 2))

savename = "./data/simulation_results/"+str(n_trials)+"_simulations_starved_large.csv"
print(ti.strftime("%a, %d %b %Y %H:%M", ti.gmtime()))
run_simulations(n_trials, widths, savename, df, chemokinesis)
elapsed = ti.time()-t0
t0 = ti.time()
print("Finished with", n_trials, "simulations for starved animals", str(elapsed), "seconds")

- Run simulation tasks using empirical data from fed animals

In [ ]:
df = pd.read_csv("./data/trajectories/summary/modeling_Fed_acclimate_all_animals.csv")
df.dropna(subset=['angle_delta', 'speed_mm_s'], inplace=True, axis=0)
df['speed'] = df['speed_mm_s']/2
print("Mean angle delta:", round(np.mean(df['angle_delta'].tolist()), 2))
print("Mean speed (mm/f):", round(np.mean(df['speed'].tolist()), 2))

savename = "./data/simulation_results/"+str(n_trials)+"_simulations_fed_large.csv"
print(ti.strftime("%a, %d %b %Y %H:%M", ti.gmtime()))
run_simulations(n_trials, widths, savename, df, chemokinesis)
elapsed = ti.time()-t0
t0 = ti.time()
print("Finished with", n_trials, "simulations for fed animals", str(elapsed), "seconds")